In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_feather("dataFrame.feather")
data["category"] = pd.Categorical(data["category"])
print(data.shape)
X = data["text"]
Y = pd.get_dummies(data["category"])
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.1)

(2225, 3)


In [3]:
import gensim

# Load Google's pre-trained Word2Vec model.
#model = gensim.models.KeyedVectors.load_word2vec_format('/home/robertb/Desktop/text_embeddings/project/GoogleNews-vectors-negative300.bin', binary=True)  
model_w2v = gensim.models.Word2Vec.load('/home/robert/Desktop/repo/text_classification/models/simple_word2vec.model')  

# get word embedding matrix
embedding_layer = model_w2v.wv.get_keras_embedding(train_embeddings=False)


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# get pairs word:index(word) 
vocabulary = {token: token_index for token_index, token in enumerate(model_w2v.wv.index2word)} 
# change words to integer seq using indexes
tk = Tokenizer(num_words=len(vocabulary)) 
tk.word_index = vocabulary 

train_text_seq = tk.texts_to_sequences(train_X)
maxlen = max([len(text) for text in train_text_seq])
train_text_seq = pad_sequences(train_text_seq, maxlen=maxlen, padding="post", value=0.0)

test_text_seq = tk.texts_to_sequences(test_X)
test_text_seq = pad_sequences(test_text_seq, maxlen=maxlen, padding="post", value=0.0)



In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Embedding, Bidirectional, Dropout, LSTM
# define the model
model_NN = tf.keras.models.Sequential()

#model_NN.add(Embedding(len(vocabulary)+1,100,trainable=True))
model_NN.add(embedding_layer)

model_NN.add(Bidirectional(LSTM(16, return_sequences=False)))

model_NN.add(Dense(5, activation='softmax'))
# compile the model
model_NN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model_NN.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         4226700   
_________________________________________________________________
bidirectional (Bidirectional (None, 32)                14976     
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 4,241,841
Trainable params: 15,141
Non-trainable params: 4,226,700
_________________________________________________________________
None


In [6]:
es = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=25, restore_best_weights=True
)
hist=model_NN.fit(train_text_seq,
                  train_Y,
                  epochs=1000,
                  batch_size=64, 
                  callbacks= es,
                  validation_data=(test_text_seq, test_Y),
                  verbose=1)

Epoch 1/1000
 7/32 [=====>........................] - ETA: 23s - loss: 1.6266 - accuracy: 0.2634

KeyboardInterrupt: 

In [8]:
import matplotlib.pyplot as plt
print(hist)
history = pd.DataFrame(hist.history)
plt.figure(figsize=(12,12));
plt.plot(history["accuracy"]);
plt.plot(history["val_accuracy"]);
plt.title("Loss with pretrained word vectors");
plt.show();

NameError: name 'hist' is not defined